## Softmax
Softmax函数的本质是将一个K维的任意实数向量，压缩（映射）成另一个K维的实数向量，其中向量中的每个元素取指都介于（0,1）范围内。
<br/>
Softmax是对逻辑回归（Logistic Regression, LR）的推广，逻辑回归用于处理二分类问题，其推广Softmax回归则用于处理多分类问题。如下图示，在数学上，Softmax函数会返回输出类的互斥概率分布，例如，网络的输出为（1,1,1,1），经过Softmax函数后输出为（0.25,0.25,0.25,0.25）。我们可以得到分类中唯一所属类别，因此通常把Softmax作为输出层的激活函数。
$$
softmax(x) = \frac{e^{x_j}}{\sum_{k=1}^{K}e^{x_k}}
$$

下面进一步举例说明。假设有一个多分类问题，但是我们只关心这些类别的最高得分的概率，那么会使用一个带有最大似然估计函数的Softmax输出层来获得所有类比输出概率的最大值。例如神经网络的分类有3个，分别为“野马”“河马”“斑马”，使用softmax作为输出层的激活函数最后只能得到一个最大的分类概率如野马（0.6），河马（0.1），斑马（0.3），其中最大值野马（0.6）。
<br/>
如果要求每次的输出都可以获得多个分类，例如希望神经网络的预测输出既像“河马”也像“野马”，那么我们不希望Softmax作为输出层，这里可以使用Sigmoid函数作为输出层的激活函数更合适，因为Sigmoid函数可以为每个类别的输出提供独立的概率。
<br/>
### 性质
$$
softmax(x) = softmax(x+c),其中c为实数
$$
Reference:
* [Softmax Function wiki](https://en.wikipedia.org/wiki/Softmax_function)
* [Classification and Loss Evaluation - Softmax and Cross Entropy Loss](https://deepnotes.io/softmax-crossentropy)
* [Word2Vec介绍：softmax函数的python实现](https://zhuanlan.zhihu.com/p/28991249)

In [1]:
import numpy as np
def softmax(x):
    """
    对输入x的每一行计算softmax
    参数：
    x: 矩阵
    返回值：
    x: 对应矩阵
    """
    origin_shape = x.shape
    
    if len(x.shape) > 1:
        # 矩阵
        tmp = np.max(x, axis=1)
        x -= tmp.reshape((x.shape[0], 1))
        x = np.exp(x)
        tmp = np.sum(x, axis = 1) # 每行求和
        x /= tmp.reshape((x.shape[0]), 1)
        
    else:
        tmp = np.max(x)
        x -= tmp
        x = np.exp(x)
        tmp = np.sum(x)
        x /= tmp
    
    return x

# Skip Gram
## 什么是skip gram
就是在已知目标单词的情况下，预测它的上下文单词。
<br/>
## 目标
计算在给定单词条件下，其他单词出现的概率。
## 词向量表示
### one-hot
one-hot就是利用$R^{|V| \times 1}$向量表示单词。其中$|V|$是单词数量。
<br/>
我们用$w_c$表示目标单词的one-hot向量。
<br/>
### word embedding
例如：
$$
v_c = \left[
        \begin{matrix}
        0.2 \\
        0.5  \\
        ...  \\
        0.1
        \end{matrix}
        \right]
$$
这是一个词向量表示。其中我们用$v_c$示目标单词的词向量。
### 单词矩阵
单词矩阵就是所有单词的词向量集合。
<br/>
主要，我们这里需要用到两个单词矩阵，一个是目标单词组成的矩阵$W \in R^{d \times V}$。另外一个矩阵由除了目标单词外的其他单词的**词向量的转置**组成的矩阵，用$W' \in R^{V \times d}$。
<br/>
另外需要说明的是，由于每一个单词都有可能作为目标单词或其他单词，因此，实际上这两个矩阵是分别包含所有单词的词向量的。
<br/>
### 单词相似度
两个单词求内积。
<br/>
### Softmax函数
我们需要知道的是softmax函数就是能够把输入转换为概率分布，也就是说使输入的实数变成分数。除此之外的内容我们暂不讨论。
$$
\sigma(z)_j = \frac{e^{z_j}}{\sum_{k=1}^{K}e^{z_k}}
\\
z = u^T_xv_c
$$

## 总结

$$
w_c : 目标单词one-hot向量
\\
v_c ： 目标单词词向量
\\
u_x : 表示除目标单词外第x个单词的词向量
\\
W \in R^{d \times V} : 目标单词矩阵
\\
W' \in R^{V \times d} : 其他单词矩阵
\\
W' = W^T
\\
d : 词向量维度
\\
V : 词汇表维度
$$

## 理解算法过程
求相似度$u^T_xv_c$步骤：
1. 求$v_c$
   $$
   v_c = Ww_c
   \\
    W \in R^{d \times V}
    \\
    w_c \in R^{V \times 1}
    v_c \in R^{d \times 1}
   $$
   求$v_c$就是在$W$中进行索引的过程，在实际操作中，采用索引进行。
2. 求$u^T_xv_c$
    <br/>
    相似度可有矩阵$W'v_c$求得。$u^T_xv_c$就是结果的每一行向量
3. 求softmax
    <br/>
    这步比较简单，把得到的相似度矩阵代入softmax公式，就得到了一个满足概率分布的矩阵。
    

至此，我们实现了我们的目标：得到一个向量。
$$
W' v_c

=

\left[
\begin{matrix}
u_1^Tv_c
\\
u_2^Tv_c
\\
u_3^Tv_c
\\
...
\\
u_V^Tv_c
\\
\end{matrix}
\right]

= 

\left[
\begin{matrix}
P(u_1 | v_c)
\\
P(u_2 | v_c)
\\
P(u_3 | v_c)
\\
...
\\
P(u_V | v_c)
\\
\end{matrix}
\right]
$$